<a href="https://colab.research.google.com/github/kailashsp/Spam_Ham_Classifier/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-10-08 10:57:29--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.8’

train-data.tsv.8    100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2022-10-08 10:57:29 (71.7 MB/s) - ‘train-data.tsv.8’ saved [358233/358233]

--2022-10-08 10:57:29--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.8’

valid-data.tsv.8    100%[========

In [ ]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization



In [ ]:
test_data = pd.read_csv('valid-data.tsv',sep = '\t',header =0, names = ['labels','messages'])

In [ ]:
train_data = pd.read_csv('train-data.tsv',sep = '\t',header =0, names = ['labels','messages'])

In [ ]:
train_data['labels'].mask(train_data['labels']=='spam',0,inplace=True)

In [ ]:
train_data['labels'].mask(train_data['labels']=='ham',1,inplace=True)

In [ ]:
texts = train_data['messages'].tolist()

In [ ]:
texts = np.array(texts)

In [ ]:
labels = train_data['labels'].tolist()

In [ ]:
labels = np.array(labels)

In [ ]:
# test_data['labels'].mask(test_data['labels']=='spam',0,inplace=True)

In [ ]:
# test_data['labels'].mask(test_data['labels']=='ham',1,inplace=True)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [ ]:
X = tokenizer.texts_to_sequences(texts)

In [ ]:
len(tokenizer.word_index)+1

7692

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X = pad_sequences(X, maxlen=30)

In [ ]:
from tensorflow.keras.layers import Embedding

In [ ]:
import tensorflow as tf

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.Input(shape=train_features.shape[1:]),
#     encoder,
#     Embedding(10000,64,mask_zero = True),
#     tf.keras.layers.SimpleRNN(32,return_sequences=True),
#     tf.keras.layers.SimpleRNN(32),
#     tf.keras.layers.Dense(32,activation='relu'),
#     tf.keras.layers.Dense(1,activation='sigmoid')
# ])

In [ ]:
input = tf.keras.Input(shape=30)

embedding = Embedding(7692,128,mask_zero = True)(input)

srnn1 = tf.keras.layers.LSTM(64,recurrent_dropout=0.2, return_sequences=True)(embedding)
srnn1 = tf.keras.layers.LSTM(64,recurrent_dropout=0.2, return_sequences=True)(embedding)
srnn1 = tf.keras.layers.LSTM(64,recurrent_dropout=0.2, return_sequences=True)(embedding)
srnn1 = tf.keras.layers.LSTM(64,recurrent_dropout=0.2, return_sequences=True)(embedding)
srnn2 = tf.keras.layers.LSTM(32)(srnn1)


dense1 = tf.keras.layers.Dense(64,activation='relu')(srnn2)

dense2 = tf.keras.layers.Dense(32,activation='relu')(dense1)

output = tf.keras.layers.Dense(1,activation='sigmoid')(dense2)

model = tf.keras.Model(inputs = [input],outputs = [output])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 embedding (Embedding)       (None, 30, 128)           984576    
                                                                 
 lstm_3 (LSTM)               (None, 30, 64)            49408     
                                                                 
 lstm_4 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33    

In [ ]:
model.compile(loss = "binary_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics ='acc')

In [ ]:
history = model.fit(X,labels,epochs=10, batch_size=16)

Epoch 1/10
262/262 [==============================] - 78s 255ms/step - loss: 0.1438 - acc: 0.9607
Epoch 2/10
262/262 [==============================] - 51s 196ms/step - loss: 0.0162 - acc: 0.9969
Epoch 3/10
262/262 [==============================] - 55s 210ms/step - loss: 0.0061 - acc: 0.9974
Epoch 4/10
262/262 [==============================] - 49s 188ms/step - loss: 5.7333e-04 - acc: 1.0000
Epoch 5/10
262/262 [==============================] - 52s 198ms/step - loss: 2.2580e-04 - acc: 1.0000
Epoch 6/10
262/262 [==============================] - 48s 185ms/step - loss: 1.7465e-04 - acc: 1.0000
Epoch 7/10
262/262 [==============================] - 49s 186ms/step - loss: 1.4752e-04 - acc: 1.0000
Epoch 8/10
262/262 [==============================] - 51s 197ms/step - loss: 1.2725e-04 - acc: 1.0000
Epoch 9/10
262/262 [==============================] - 48s 185ms/step - loss: 1.1107e-04 - acc: 1.0000
Epoch 10/10
262/262 [==============================] - 48s 183ms/step - loss: 9.7788e-05 - acc

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  sequence = tokenizer.texts_to_sequences([pred_text])
    # pad the sequence
  sequence = pad_sequences(sequence, maxlen=30)
    # get the prediction
  predict = model.predict(sequence)[0]
  pred = list(predict)

  if predict > 0.5:
    pred.append('ham')
  else:
    pred.append('spam')
  
  predicts = pred

  return (predicts)

pred_text =  "sale today! to stop texts call 98912460324"
                  

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 1s 1s/step
[0.49274388, 'spam']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                  "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham","spam","ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 36ms/step
You passed the challenge. Great job!
